# Analysing the scalarizings when using the surrogate (600/6)

In [49]:
%matplotlib inline
%pylab inline 
import seaborn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os
from ASF import ASF, NIMBUS
from gradutil import *
from pyomo.opt import SolverFactory
from scipy.spatial.distance import euclidean
from interactiveBoreal import ReferenceFrame, Solver

In [2]:
nclust = 600
seedn = range(2,12)[4] # == 6

In [3]:
kehys = ReferenceFrame()
#kehys.cluster(nclust=nclust, seedn=seedn)

In [50]:
with open('clusterings/new_{}.json'.format(nclust), 'r') as file:
    clustering = json.load(file)
kehys.xtoc = np.array(clustering[str(seedn)]['xtoc'])
kehys.weights = np.array([sum(kehys.xtoc == i)
                          for i in range(nclust)
                          if sum(kehys.xtoc == i) > 0])
indices = [min(np.array(range(len(kehys.xtoc)))[kehys.xtoc == i],
               key=lambda index: euclidean(kehys.x_norm[index],
                                           np.mean(kehys.x_norm[kehys.xtoc == i], axis=0)))
           for i in range(nclust) if sum(kehys.xtoc == i) > 0]
kehys.centers = kehys.x_stack[indices]
kehys.out_centers = kehys.x_stack[indices]

In [51]:
names = np.array(['ASF', 'STOM', 'GUESS', 'NIMBUS'])
reg_names = np.array(['BAU','SA', 'EXT10','EXT30','GTR30','NTSR','NTL'])
revenue,_,_,_ = init_boreal()
data = kehys.centers
nobj = np.shape(data)[-1]
nvar = len(kehys.x_norm)
weights = kehys.weights/nvar
norm_ideal = kehys.normalize_ref(kehys.ideal)
norm_nadir = kehys.normalize_ref(kehys.nadir)
solver_name = 'cplex'

In [52]:
solver = SolverFactory('cplex')
problems = []
ress = []

for i in range(nobj):
    problems.append(BorealWeightedProblem(data[:, :, i], weights, nvar))
    
for p in problems:
    ress.append(solver.solve(p.model))
    
payoff = [[cluster_to_value(kehys.out_centers[:,:,i], res_to_list(p.model), kehys.weights) for i in range(nobj)] for p in problems]
ide_surr = np.max(payoff, axis=0)
nad_surr = np.min(payoff, axis=0)

payoff_model = [[model_to_real_values(kehys.x_stack[:, :, i], p.model, kehys.xtoc) for i in range(nobj)] for p in problems]
ide_orig = np.max(payoff_model, axis=0)
nad_orig = np.min(payoff_model, axis=0)

In [53]:
for r in ress:
    print(r['Solver'][0]['Termination message'])

In [54]:
for p in payoff:
    for f in p:
        print('{:11.1f}'.format(f), end=' ')
    print('')

In [9]:
ide_surr, ide_orig, kehys.ideal

In [10]:
nad_surr, nad_orig, kehys.nadir

## Testing scalarizations for surrogate (real ideal and nadir)

In [40]:
def scalarizings(init_ref, maxs=[], frees=[], output=False):
    ref = init_ref
    stay = np.array([], dtype=int)
    detoriate = np.array([], dtype=int)
    minmax = np.array(maxs, dtype=int)
    
    asf   = ASF(kehys.ideal, kehys.nadir, ref, data, weights=weights, nvar=nvar, scalarization='asf',   sense='maximize')
    stom  = ASF(kehys.ideal, kehys.nadir, ref, data, weights=weights, nvar=nvar, scalarization='stom',  sense='maximize')
    guess = ASF(kehys.ideal, kehys.nadir, ref, data, weights=weights, nvar=nvar, scalarization='guess', sense='maximize', frees=frees)
    
    asf_solver = Solver(asf.model, solver=solver_name)
    asf_res = asf_solver.solve()
    asf_values = kehys.values(model=asf.model)
    
    stom_solver = Solver(stom.model, solver=solver_name)
    stom_res = stom_solver.solve()
    stom_values = kehys.values(model=stom.model)

    guess_solver = Solver(guess.model, solver=solver_name)
    guess_res = guess_solver.solve()
    guess_values = kehys.values(model=guess.model)
    
    nimbus = NIMBUS(kehys.ideal, kehys.nadir, ref, data, minmax, stay, detoriate, asf_values, weights=weights, nvar=nvar)
    nimbus_solver = Solver(nimbus.model, solver=solver_name)
    nimbus_res = nimbus_solver.solve()
    nimbus_values = kehys.values(model=nimbus.model)
    
    if output:
        problems = [asf, stom, guess, nimbus]
        listss = [res_to_list(pro.model) for pro in problems]
        orig_stands = revenue.values
        all_regs = []
        for l in listss:
            nos = dict()
            for ind,n in enumerate(l):
                these_stands_to_bau = orig_stands[kehys.xtoc==ind,int(n)]
                to_bau_no = np.sum(np.isnan(these_stands_to_bau))
                nos[0] = nos.get(0, 0) + to_bau_no
                nos[int(n)] = nos.get(n, 0) + kehys.weights[ind] - to_bau_no
            all_regs.append(nos)

        for i,di in enumerate(all_regs):
            print('\n{}'.format(names[i]))
            summ = 0
            for key in di.keys():
                summ += di[key]
                print('{:5} {:6}'.format(reg_names[key], di[key]))
            print('Total: {}'.format(summ))
        
    return asf_values, stom_values, guess_values, nimbus_values

In [48]:
    hectars = 68700
    starting_solution = np.array((2860, 56.2, 2.86,  0.26))*hectars
    ref = np.array((kehys.nadir[0], 51*hectars, 2*hectars, kehys.ideal[3]))
    stay = np.array([], dtype=int)
    detoriate = np.array([1,2], dtype=int)
    minmax = np.array([3], dtype=int)
    nimbus = NIMBUS(kehys.ideal, kehys.nadir, ref, data, minmax, stay, detoriate, starting_solution, weights=weights, nvar=nvar)
    nimbus_solver = Solver(nimbus.model, solver=solver_name)
    nimbus_res = nimbus_solver.solve()
    nimbus_values = kehys.values(model=nimbus.model)
    print(['{:11.2f}'.format(value) for value in nimbus_values/hectars])

### Revenue to Ideal

In [41]:
init_ref = np.array((kehys.ideal[0], kehys.nadir[1], kehys.nadir[2], kehys.nadir[3]))
res1 =  scalarizings(init_ref, maxs=[0], frees=[1,2,3])

In [42]:
for f in res1:
    print(['{:11.2f}'.format(value) for value in f])

In [14]:
print(['{:11.2f}'.format(value) for value in kehys.ideal])

### Carbon to Ideal

In [43]:
init_ref = np.array((kehys.nadir[0], kehys.ideal[1], kehys.nadir[2], kehys.nadir[3]))
res2 = scalarizings(init_ref, maxs=[1], frees=[0,2,3])

In [16]:
for f in res2:
    print(['{:12.2f}'.format(value) for value in f])

In [17]:
print(['{:11.2f}'.format(value) for value in kehys.ideal])

### Deadwood to Ideal

In [18]:
init_ref = np.array((kehys.nadir[0], kehys.nadir[1], kehys.ideal[2], kehys.nadir[3]))
res3 = scalarizings(init_ref, maxs=[2], frees=[0,1,3])

In [19]:
for f in res3:
    print(['{:12.2f}'.format(value) for value in f])

In [20]:
print(['{:11.2f}'.format(value) for value in kehys.ideal])

### Habitat to Ideal

In [21]:
init_ref = np.array((kehys.nadir[0], kehys.nadir[1], kehys.nadir[2], kehys.ideal[3]))
res4= scalarizings(init_ref, maxs=[3], frees=[0,1,2])

In [22]:
for f in res4:
    print(['{:12.2f}'.format(value) for value in f])

In [23]:
print(['{:11.2f}'.format(value) for value in kehys.ideal])

In [100]:
print(res4[-1])

## Ideal and nadir from scalarizings

In [24]:
all_res = np.concatenate((res1, res2, res3, res4))
all_arr = np.array((res1, res2, res3, res4))

In [25]:
objs_long = np.array(('Timber Revenue','Carbon Storage','Deadwood Volume','Habitat Suitability'))
objs = np.array(('Revenue','Carbon','Deadwood','Habitat'))
names = np.array(('Achievement Scalarizing Function','Satisficing Trade-Off Method','GUESS-scalarizing','NIMBUS-scalarizing'))
for obj,obj_res in zip(objs_long,all_arr):
    print(' & \\multicolumn{{4}}{{|c|}}{{{}}}\\\\'.format(obj))
    print(' & {} \\\\'.format(' & '.join(objs)))
    for name, scal_res in zip(names,obj_res):
        print('{} & {} \\\\'.format(name, ' & '.join(['{:,.0f}'.format(value).replace(',', ' ') for value in scal_res])))
    print('\hline')

In [26]:
scal_ideal = np.max(all_res, axis=0)
scal_nadir = np.min(all_res, axis=0)

In [27]:
print(['{:12.0f}'.format(value) for value in scal_ideal])
print(['{:12.0f}'.format(value) for value in scal_nadir])

In [28]:
print(['{:12.0f}'.format(value) for value in ide_surr])
print(['{:12.0f}'.format(value) for value in nad_surr])

In [29]:
print(' & '.join(['{:,.0f}'.format(value).replace(',', ' ') for value in ide_surr]))
print(' & '.join(['{:,.0f}'.format(value).replace(',', ' ') for value in nad_surr]))

In [30]:
print(' & '.join(['{:,.0f}'.format(value).replace(',', ' ') for value in kehys.ideal]))
print(' & '.join(['{:,.0f}'.format(value).replace(',', ' ') for value in kehys.nadir]))

## Testing scalarizations for surrogate (ideal and nadir from the surrogate)

In [78]:
def scalarizings_surr(init_ref, maxs=[], frees=[]):
    stay = np.array([], dtype=int)
    detoriate = np.array([], dtype=int)
    minmax = np.array(maxs, dtype=int)
    
    asf   = ASF(ide_surr, nad_surr, ref, data, weights=weights, nvar=nvar, scalarization='asf', sense='maximize')
    stom  = ASF(ide_surr, nad_surr, ref, data, weights=weights, nvar=nvar, scalarization='stom', sense='maximize')
    guess = ASF(ide_surr, nad_surr, ref, data, weights=weights, nvar=nvar, scalarization='guess', sense='maximize', frees=frees)
    
    asf_solver = Solver(asf.model, solver=solver_name)
    asf_res = asf_solver.solve()
    asf_values = kehys.values(model=asf.model)
    
    stom_solver = Solver(stom.model, solver=solver_name)
    stom_res = stom_solver.solve()
    stom_values = kehys.values(model=stom.model)

    guess_solver = Solver(guess.model, solver=solver_name)
    guess_res = guess_solver.solve()
    guess_values = kehys.values(model=guess.model)
    
    #Use tha ASF value as initial value for the NIMBUS. NOTICE THE DIFFERENT WEIGTHS ALSO!
    nimbus = NIMBUS(ide_surr, nad_surr, ref, data, minmax, stay, detoriate, asf_values, weights=weights, nvar=nvar)
    nimbus_solver = Solver(nimbus.model, solver=solver_name)
    nimbus_res = nimbus_solver.solve()
    nimbus_values = kehys.values(model=nimbus.model)

    return asf_values, stom_values, guess_values, nimbus_values

### Revenue to Ideal

In [79]:
init_ref = np.array((kehys.ideal[0], kehys.nadir[1], kehys.nadir[2], kehys.nadir[3]))
surr_res1 =  scalarizings_surr(init_ref, maxs=[0], frees=[1,2,3])

In [80]:
for f in surr_res1:
    print(['{:11.2f}'.format(value) for value in f])

In [81]:
print(['{:11.2f}'.format(value) for value in kehys.ideal])

### Carbon to Ideal

In [82]:
init_ref = np.array((kehys.nadir[0], kehys.ideal[1], kehys.nadir[2], kehys.nadir[3]))
surr_res2 = scalarizings_surr(init_ref, maxs=[1], frees=[0,2,3])

In [83]:
for f in surr_res2:
    print(['{:12.2f}'.format(value) for value in f])

In [84]:
print(['{:11.2f}'.format(value) for value in kehys.ideal])

### Deadwood to Ideal

In [85]:
init_ref = np.array((kehys.nadir[0], kehys.nadir[1], kehys.ideal[2], kehys.nadir[3]))
surr_res3 = scalarizings_surr(init_ref, maxs=[2], frees=[0,1,3])

In [86]:
for f in surr_res3:
    print(['{:11.2f}'.format(value) for value in f])

In [87]:
print(['{:11.2f}'.format(value) for value in kehys.ideal])

### Habitat to Ideal

In [88]:
init_ref = np.array((kehys.nadir[0], kehys.nadir[1], kehys.nadir[2], kehys.ideal[3]))
surr_res4= scalarizings_surr(init_ref, maxs=[3], frees=[0,1,2])

In [89]:
for f in surr_res4:
    print(['{:11.2f}'.format(value) for value in f])

In [90]:
print(['{:11.2f}'.format(value) for value in kehys.ideal])

## Ideal and nadir from scalarizings

In [91]:
surr_all_res = np.concatenate((surr_res1, surr_res2, surr_res3, surr_res4))
surr_all_arr = np.array((surr_res1, surr_res2, surr_res3, surr_res4))

In [92]:
objs_long = np.array(('Timber Revenue','Carbon Storage','Deadwood Volume','Habitat Suitability'))
objs = np.array(('Revenue','Carbon','Deadwood','Habitat'))
names = np.array(('Achievement Scalarizing Function','Satisficing Trade-Off Method','GUESS-scalarizing','NIMBUS-scalarizing'))
for obj,obj_res in zip(objs_long,surr_all_arr):
    print(' & \\multicolumn{{4}}{{|c|}}{{{}}}\\\\'.format(obj))
    print(' & {} \\\\'.format(' & '.join(objs)))
    for name, scal_res in zip(names,obj_res):
        print('{} & {} \\\\'.format(name, ' & '.join(['{:,.0f}'.format(value).replace(',', ' ') for value in scal_res])))
    print('\hline')

In [93]:
surr_scal_ideal = np.max(surr_all_res, axis=0)
surr_scal_nadir = np.min(surr_all_res, axis=0)

In [94]:
print(['{:12.0f}'.format(value) for value in surr_scal_ideal])
print(['{:12.0f}'.format(value) for value in surr_scal_nadir])

In [95]:
print(['{:12.0f}'.format(value) for value in scal_ideal])
print(['{:12.0f}'.format(value) for value in scal_nadir])

In [96]:
print(['{:12.0f}'.format(value) for value in ide_surr])
print(['{:12.0f}'.format(value) for value in nad_surr])